In [ ]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Function
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from model.dataset import SMALRandomDataset

from model.SkinningNet import SkinningNet
from model.utils import SkinningLoss, getSkeletonOneRingIdx

from meshplot import plot
import igl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 8
dataset = SMALRandomDataset()
trainIter = DataLoader(dataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTrainNUM = len(dataset)

In [ ]:
net = SkinningNet(jointNum = 33)
#net.load_state_dict(torch.load(".\stateDict\skinningNet_finetune_noise.pkl"))
net = net.to(device)

In [ ]:
lr = 0.00005
epochs = 200
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=lr)
loss = SkinningLoss()

In [ ]:
beginTime = time.time()
net = net.train()

for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        joints = joints.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        
        opt.zero_grad()

        preJ, preA = net(V, facesOneRingIdx, rigW)
        jl, sl, acc = loss(preJ, preA, joints, rigW.permute(0, 2, 1))
        l = jl+sl
        l.backward()
        opt.step()
        
        batchLoss = float(jl)
        batchAcc = float(acc)
        trainAcc += batchAcc
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r 训练%i / %i, 关节点损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTrainNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
        sys.stdout.flush()
    torch.save(net.state_dict(), ".\stateDict\skinningNet_%d.pkl"%epoch)
    print("\n")
    print("epoch%d, 关节点损失: %f, 准确率：%0.2f%%, 用时：%0.2f minuetes"%(epoch+1, trainLoss/batchNum, trainAcc/batchNum*100, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step() 

In [ ]:
lr = 0.00005
epochs = 50
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=lr)
loss = SkinningLoss()

In [ ]:

beginTime = time.time()
net = net.train()

for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        joints = joints.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        
        opt.zero_grad()

        preJ, preA = net(V, facesOneRingIdx)
        jl, sl, acc = loss(preJ, preA, joints, rigW.permute(0, 2, 1))
        l = jl+sl
        l.backward()
        opt.step()
        
        batchLoss = float(jl)
        batchAcc = float(acc)
        trainAcc += batchAcc
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r 训练%i / %i, 关节点损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTrainNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
        sys.stdout.flush()
    torch.save(net.state_dict(), ".\stateDict\skinningNet_%d.pkl"%epoch)
    print("\n")
    print("epoch%d, 关节点损失: %f, 准确率：%0.2f%%, 用时：%0.2f minuetes"%(epoch+1, trainLoss/batchNum, trainAcc/batchNum*100, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step() 